In [1]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd

# 環境変数の取得
load_dotenv()

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [2]:
# 1. Excelファイルを読み込む
df = pd.read_excel('サンプルデータ.xlsx', sheet_name='売上データ')
# データフレームを表示して確認
df.head()

,カテゴリー,商品コード,商品名,売上日,単価,数量,原価
0,食品,1001,りんご,2023-01-01,200,50,120
1,食品,1002,バナナ,2023-01-01,150,100,80
2,食品,1003,牛乳,2023-01-02,180,80,100
3,衣服,2001,Tシャツ,2023-01-02,1500,20,800
4,衣服,2002,ジーンズ,2023-01-03,5000,10,2500


In [3]:
# 2. データをLLM用にテキスト形式に変換
# データフレーム全体を文字列に変換
sales_data_text = df.astype(str)
prompt_text = f"売上データ:\n{sales_data_text}\nこの売上データの傾向を分析してください。"
# 表示して確認
print(prompt_text)

売上データ:
    カテゴリー 商品コード      商品名         売上日    単価   数量    原価
0      食品  1001      りんご  2023-01-01   200   50   120
1      食品  1002      バナナ  2023-01-01   150  100    80
2      食品  1003       牛乳  2023-01-02   180   80   100
3      衣服  2001     Tシャツ  2023-01-02  1500   20   800
4      衣服  2002     ジーンズ  2023-01-03  5000   10  2500
..    ...   ...      ...         ...   ...  ...   ...
235    衣服  2077   レインパンツ  2023-04-28  2000   18  1000
236    食品  1085      ザクロ  2023-04-29   600   40   300
237   日用品  3077    バスブラシ  2023-04-29   400   60   200
238    衣服  2078  レインシューズ  2023-04-30  2500   15  1250
239    食品  1086    ココナッツ  2023-04-30   300   80   150

[240 rows x 7 columns]
この売上データの傾向を分析してください。


In [4]:
# 3. OpenAI APIの呼び出し

# 役割を設定
role = "あなたはマーケティング分野に精通したデータサイエンティストです。企業の成長をサポートするために、効果的なインサイトを提供します。"

# APIへリクエスト
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": role},
        {"role": "user", "content": prompt_text},
    ],
)

# LLMからの回答を表示
print(response.choices[0].message.content.strip())

この売上データを分析するために、以下の主要な観点から傾向を確認します。

### 1. 売上の全体的な傾向
- **カテゴリー別売上**: 食品、衣服、日用品の各カテゴリーでの売上の割合を集計し、どのカテゴリーが総売上に寄与しているかを分析します。
- **時間による変動**: 売上日別に売上を集計し、時間の経過とともに売上がどのように変動しているかを確認します（例：月ごとのトレンド）。

### 2. 商品別のパフォーマンス
- **売上の高い商品**: 各商品の総売上（単価 × 数量）を計算し、どの商品が最も売上を上げているのか、または利益をもたらしているのかを特定します。
- **原価と利益**: 各商品の原価も考慮し、売上総利益（単価 - 原価）を計算することで、各商品の利益率を見ます。

### 3. 人気のある商品や季節性
- **数量トレンド**: 売上数量が多い商品や、特定の季節やイベントに関連して売上が急増した商品を特定します。
- **季節ごとの需要**: 特定の季節や月に人気が集中する商品があるかどうかを調査します。

### 4. 顧客の購買行動
- **リピート購入の分析**: 売上日を基に、ある商品がリピーターからどれだけ購入されているか、顧客の購買サイクルについて分析します。

### 5. 改善点と機会
- **在庫管理の最適化**: 売上データに基づいて、どの商品が過剰在庫または在庫不足の可能性があるかを見極め、在庫戦略を最適化します。
- **マーケティング戦略の提案**: 履歴データを基に、どのカテゴリーや商品のマーケティングを強化するべきか、プロモーション戦略の提案を行います。

これらの分析を通じて、データを可視化し、洞察を得ることで、ビジネスの成長に寄与する戦略や施策を提案することが可能となります。具体的な数値やトレンドについては、さらに詳細なデータの集計や可視化が必要ですが、上記の観点をもとに詳細な分析を進めることが理想です。


In [5]:
# 4. 分析結果をデータフレームに変換
result_list = response.choices[0].message.content.strip().split("\n")
df_out = pd.DataFrame(result_list, columns=['結果'])
print(df_out)


                                                   結果
0                 この売上データを分析するために、以下の主要な観点から傾向を確認します。
1                                                    
2                                    ### 1. 売上の全体的な傾向
3   - **カテゴリー別売上**: 食品、衣服、日用品の各カテゴリーでの売上の割合を集計し、どの...
4   - **時間による変動**: 売上日別に売上を集計し、時間の経過とともに売上がどのように変動...
5                                                    
6                                  ### 2. 商品別のパフォーマンス
7   - **売上の高い商品**: 各商品の総売上（単価 × 数量）を計算し、どの商品が最も売上を...
8   - **原価と利益**: 各商品の原価も考慮し、売上総利益（単価 - 原価）を計算することで...
9                                                    
10                                 ### 3. 人気のある商品や季節性
11  - **数量トレンド**: 売上数量が多い商品や、特定の季節やイベントに関連して売上が急増し...
12     - **季節ごとの需要**: 特定の季節や月に人気が集中する商品があるかどうかを調査します。
13                                                   
14                                     ### 4. 顧客の購買行動
15  - **リピート購入の分析**: 売上日を基に、ある商品がリピーターからどれだけ購入されてい...
16                                                   
17                          

In [6]:
# 5. 結果をExcelファイルに保存
df_out.to_excel("売上データ分析結果.xlsx", index=False)

In [7]:
# ワークフロー化
print("処理を開始します。")

# 1. Excelファイルを読み込む
df = pd.read_excel('サンプルデータ.xlsx', sheet_name='売上データ')

# 2. データをLLM用にテキスト形式に変換
sales_data_text = df.astype(str)
prompt_text = f"売上データ:\n{sales_data_text}\nこの売上データの傾向を分析してください。"

# 3. OpenAI APIの呼び出し
# 役割を設定
role = "あなたはマーケティング分野に精通したデータサイエンティストです。企業の成長をサポートするために、効果的なインサイトを提供します。"
# APIへリクエスト
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": role},
        {"role": "user", "content": prompt_text},
    ],
)

# 4. 分析結果をデータフレームに変換
result_list = response.choices[0].message.content.strip().split("\n")
df_out = pd.DataFrame(result_list, columns=['結果'])

# 5. 結果をExcelファイルに保存
df_out.to_excel("売上データ分析結果.xlsx", index=False)

print("Excelファイルに分析結果を保存しました。")

処理を開始します。
Excelファイルに分析結果を保存しました。
